
Using the best AD input parameters found by the Find_Events.ipynb script,
load the results, find the corresponding start and end times for runoff events,
query for any missing radar images, and batch mask the images to yield
just the pixels representing each station catchment.




In [2]:
import pandas as pd
import numpy as np
import os 
import sys
import math
import utm
import time

import json
import geopandas as gpd
import fiona
from geopy import distance

from numba import jit

import codecs

from shapely.geometry import Point, shape, mapping, Polygon

from sklearn.decomposition import PCA
from sklearn import preprocessing

from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from bokeh.plotting import ColumnDataSource, output_notebook
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral3
from bokeh.layouts import gridplot

import matplotlib.pyplot as plt

from radar_scrape import get_radar_img_urls, request_img_files
from get_station_data import get_daily_runoff

from numpy.random import seed
import tensorflow

from keras.layers import Input, Dropout
from keras.layers.core import Dense 
from keras.models import Model, Sequential, load_model
from keras import regularizers
from keras.models import model_from_json

from radar_station_coords import radar_sites

output_notebook()
%matplotlib inline

Loading BokehJS ...

In [3]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
DB_DIR = os.path.join(BASE_DIR, 'code/hydat_db')
PROJECT_DIR = os.path.abspath('')
# IMG_DIR = os.path.join(PROJECT_DIR, 'data/radar_img')
RADAR_IMG_DIR = os.path.join(PROJECT_DIR, 'data/sorted_radar_images')

RESULTS_DIR = os.path.join(PROJECT_DIR, 'data/AD_results')


In [4]:
def find_closest_radar_stn(row):
    """ 
    Input the dict of all station distances,
    Return the location code of the nearest radar station.
    """
    radar_station_distances = row['radar_stn_distance_dict']
    min_dist = min(radar_station_distances.items(), key=lambda x: x[1])
    return min_dist[0]


def find_closest_radar_stn_distance(row):
    """ 
    Input the dict of all station distances,
    Return the location code of the nearest radar station.
    """
    radar_station_distances = row['radar_stn_distance_dict']
    min_dist = min(radar_station_distances.items(), key=lambda x: x[1])
    return min_dist[1]

def calc_distance(wsc_row, station):
    wsc_stn_coords = (wsc_row['Latitude'], wsc_row['Longitude'])
    radar_coords = radar_sites[station]['lat_lon']
    return distance.distance(radar_coords, wsc_stn_coords).km

def calculate_radar_stn_distances(row):
    distance_dict = {}
    for site in radar_sites:
        distance_dict[site] = calc_distance(row, site)
    return distance_dict

In [5]:
def initialize_wsc_station_info_dataframe():
    # import master station list
    stations_df = pd.read_csv(DB_DIR + '/WSC_Stations_Master.csv')
    # filter for stations that have concurrent record with the historical radar record
    stations_df['RADAR_Overlap'] = stations_df['Year To'].astype(int) - 2007
    stations_filtered = stations_df[stations_df['RADAR_Overlap'] > 0]
    # filter for stations that are natural flow regimes
    stations_filtered = stations_filtered[stations_filtered['Regulation'] == 'N']
    stations_filtered.rename(columns={'Gross Drainage Area (km2)': 'DA'}, inplace=True)
    # filter for stations in Alberta and British Columbia
    stations_filtered = stations_filtered[(stations_filtered['Province'] == 'BC') | (stations_filtered['Province'] == 'AB')]
    
    # calculate distance to each radar station
    stations_filtered['radar_stn_distance_dict'] = stations_filtered.apply(lambda row: calculate_radar_stn_distances(row), axis=1)    
    stations_filtered['closest_radar_station'] = stations_filtered.apply(lambda row: find_closest_radar_stn(row), axis=1)
    stations_filtered['radar_distance_km'] = stations_filtered.apply(lambda row: find_closest_radar_stn_distance(row), axis=1)
    
    # radar range is a 240km radius from the station
    stations_filtered = stations_filtered[stations_filtered['radar_distance_km'] < 211]
    stn_df = stations_filtered[np.isfinite(stations_filtered['DA'].astype(float))]
    # filter for stations greater than 10 km^2 (too small for meaningful results)
    stn_df = stn_df[stn_df['DA'].astype(float) >= 10]
    # filter for stations smaller than 1000 km^2 (too large and complex)
    stn_df = stn_df[stn_df['DA'].astype(float) < 1000].sort_values('DA')
    df = stn_df[['Province', 'Station Number', 'Station Name', 'DA', 
                 'Elevation', 'Latitude', 'Longitude', 'RADAR_Overlap',
                'closest_radar_station', 'radar_stn_distance_dict', 'radar_distance_km']]
#     print('After filtering, there are {} candidate stations.'.format(len(stn_df)))
    df.reset_index(inplace=True)
    return df


In [6]:
stn_df = initialize_wsc_station_info_dataframe()

In [7]:
print('After filtering, there are {} candidate stations.'.format(len(stn_df)))

After filtering, there are 168 candidate stations.


In [8]:
# retrieve the results dataframe for the specified site
results_folders = os.listdir(RESULTS_DIR)

results_dict = {}
# create a dictionary of results from all AD searches
for f in results_folders:
    folder_path = os.path.join(RESULTS_DIR, f)
    all_sites = [e.split('_')[0] for e in os.listdir(folder_path)]
    for site in all_sites:
        if site in results_dict.keys():
            old_results = results_dict[site]
            new_results = pd.read_csv(os.path.join(folder_path, site + '_results.csv'))
            results_dict[site] = pd.concat([old_results, new_results], sort=True)
        else:            
            results_dict[site] = pd.read_csv(os.path.join(folder_path, site + '_results.csv'))


def get_best_result(site):
    ad_df = pd.DataFrame(results_dict[site])
    ad_df.drop(labels='Unnamed: 0', inplace=True, axis=1)
    ad_df.sort_values('len_results', inplace=True, ascending=False)
    return ad_df.iloc[0, :]

all_sites = list(results_dict.keys())

In [9]:
# get all wsc_catchment data into its own dataframe
gdb_path = os.path.join(DB_DIR, 'WSC_Basins.gdb.zip')
all_layers = fiona.listlayers(gdb_path)
all_layer_names = [e.split('_')[1].split('_')[0] for e in all_layers]
filtered_layers = list(set(all_sites).intersection(all_layer_names))
# some basins don't have geometry

In [11]:
basin_df = gpd.GeoDataFrame()
i = 1
for layer in filtered_layers:
    layer_label = 'EC_' + layer + '_1'
    if i % 10 == 0:
        print('layer {} of {}: {}'.format(i, len(filtered_layers), layer))
    s = gpd.read_file(gdb_path, driver='FileGDB', layer=layer_label)
    basin_df = basin_df.append(s, ignore_index=True)
    i += 1
    
# print(basin_df.head())

layer 10 of 108: 05BM018
layer 20 of 108: 05CE012
layer 30 of 108: 08KA001
layer 40 of 108: 08GA072
layer 50 of 108: 05FA014
layer 60 of 108: 05AB040
layer 70 of 108: 08NN015
layer 80 of 108: 05CA002
layer 90 of 108: 08LF094
layer 100 of 108: 08NE008


In [12]:
basin_path = os.path.join(PROJECT_DIR, 'data/basin_geometry_data.geojson')
basin_df.to_file(basin_path, driver='GeoJSON')

In [ ]:
stn_df = stn_df[stn_df['Station Number'].isin(list(results_dict.keys()))]

In [ ]:
test_stn = stn_df['Station Number'].values[0]

best_result = get_best_result(test_stn)

test_stn_info = stn_df[stn_df['Station Number'] == test_stn]

test_stn_name = test_stn_info['Station Name'].values[0]
test_stn_da = test_stn_info['DA'].values[0]
print('Station {} ({}) has a DA of {} km^2'.format(test_stn, test_stn_name, test_stn_da))

In [ ]:
df = get_daily_runoff(test_stn)
df['Year'] = df.index.year
df['Month'] = df.index.month

In [ ]:
# filter by minimum radar date
df = df[df.index > pd.to_datetime('2007-05-31')]

In [ ]:
df['Date'] = df.index.values

In [ ]:
df[df['Year'] > 2013].plot('Date', 'DAILY_FLOW')

In [ ]:
# from peak_detection import find_peaks

In [ ]:
def find_peaks(data, lag=7, threshold=500, influence=0.5):
    # Settings (the ones below are examples: choose what is best for your data)
#     lag = 5         # lag 5 for the smoothing functions
#     threshold = 3.5  # 3.5 standard deviations for signal
#     influence = 0.5  # between 0 and 1, where 1 is normal influence, 0.5 is half
    # Initialize variables
    signals = np.zeros(len(data))            # Initialize signal results
    filteredY = np.empty(len(data))
    filteredY[:lag] = data[:lag]             # Initialize filtered series
    avgFilter = [0]                          # Initialize average filter
    stdFilter = [0]                          # Initialize std. filter
    avgFilter = {lag: np.mean(data[:lag])}      # Initialize first value
    stdFilter = {lag: np.std(data[:lag])}     # Initialize first value
    
    for i in range(lag + 1, len(data)):
        d = data[i]
        
        af = avgFilter[i-1]
        sf = stdFilter[i-1]
        
        if abs(d - af) > threshold * sf:
            if d > af:
                signals[i] = 1                     # Positive signal
            else:
                signals[i] = -1                    # Negative signal

            
            filteredY[i] = influence*d + (1-influence)*filteredY[i-1]
        else:
            signals[i] = 0                        # No signal
            filteredY[i] = 0
        
        
        # Adjust the filters
        avgFilter[i] = np.mean(filteredY[i-lag:i])
        stdFilter[i] = np.std(filteredY[i-lag:i])
        
    return signals, filteredY

n_test = 500

dats = list(df['DAILY_FLOW'].to_numpy())
sigs, f_dat = find_peaks(dats, influence=0.75, lag=7, threshold=5)

In [ ]:
from bokeh.plotting import figure, output_file, show, output_notebook

input_sig = df[['DAILY_FLOW']].copy()
signal = np.array(sigs)
input_sig['sig'] = signal.copy().astype(int)
input_sig['f_sig'] = f_dat

foo = input_sig[input_sig['sig'] == 1].copy()
p = figure(plot_width=800, plot_height=400, x_axis_type='datetime')
# add a circle renderer with a size, color, and alpha
p.circle(foo.index, foo['DAILY_FLOW'], size=10, color="red", 
         alpha=0.5, legend_label='{} pts'.format(len(foo)))
# p.line(input_sig.index, input_sig['f_sig'], color='blue')
p.line(input_sig.index, input_sig['DAILY_FLOW'], color='blue')
# p.line()
# show the results
show(p)

## Anomaly Detection Code

Adapted from [Anomaly Detection ML Methods article](https://towardsdatascience.com/machine-learning-for-anomaly-detection-and-condition-monitoring-d4614e7de770)

In [ ]:
def create_lag_df(flow_df, stn_da):

    lag_df = flow_df.copy()
    
    lag_df.rename(columns={'DAILY_FLOW': 'Q'}, inplace=True)

    num_lags = int(np.ceil(stn_da / 100) + 5)

    for i in range(1,num_lags):
        lag_df['Q{}'.format(i)] = lag_df['Q'].shift(i)

    lag_df.dropna(inplace=True)
    
    return lag_df, num_lags

In [ ]:
def split_train_and_test_data(data, training_months, training_year):
    time_range_check = (data.index.year == training_year) & (data.index.month.isin(training_months))
    train_data = data[time_range_check]
    # the test data is the entire dataset because we want to extract
    # extreme events from the training year as well
    test_data = data
    return train_data, test_data

In [ ]:
def MahalanobisDist(inv_cov_matrix, mean_distr, data, verbose=False):
    inv_covariance_matrix = inv_cov_matrix
    vars_mean = mean_distr
    diff = data - vars_mean
    md = []
    for i in range(len(diff)):
        md.append(np.sqrt(diff[i].dot(inv_covariance_matrix).dot(diff[i])))
    return md

def MD_detectOutliers(dist, extreme=False, verbose=False):
    k = 3. if extreme else 2.
    threshold = np.mean(dist) * k
    outliers = []
    for i in range(len(dist)):
        if dist[i] >= threshold:
            outliers.append(i)  # index of the outlier
    return np.array(outliers)

def MD_threshold(dist, extreme=False, verbose=False):
    k = 3. if extreme else 2.
    threshold = np.mean(dist) * k
    return threshold

def is_pos_def(A):
    if np.allclose(A, A.T):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False

In [ ]:
def initialize_runoff_dataframe(test_stn):
    runoff_df = get_daily_runoff(test_stn)
    runoff_df['Q'] = runoff_df['DAILY_FLOW']
    runoff_df = runoff_df[['Q']]
    
    # filter by minimum radar date
    runoff_df = runoff_df[runoff_df.index > pd.to_datetime('2007-05-31')]
  
    return runoff_df


In [ ]:
def do_PCA(X_train, X_test, n_components):
    
    for n_components_kept in range(2, n_components + 1):

        pca = PCA(n_components=n_components_kept, svd_solver= 'full')
        X_train_PCA = pca.fit_transform(X_train)
        X_train_PCA = pd.DataFrame(X_train_PCA)
        X_train_PCA.index = X_train.index

        X_test_PCA = pca.transform(X_test)
        X_test_PCA = pd.DataFrame(X_test_PCA)
        X_test_PCA.index = X_test.index

        var_expl = 100*np.sum(pca.explained_variance_ratio_)
        if var_expl >= 90:
#             print('var > 0.9 in {} components'.format(n_components_kept))
            return X_train_PCA, X_test_PCA, var_expl, n_components_kept
#     print('var < 0.9 in {} components'.format(n_components_kept))
    return X_train_PCA, X_test_PCA, var_expl, n_components_kept

In [ ]:
def get_start_from_annual_distribution(df):
    annual_dist = df.groupby(df.index.month).mean()
#     print(annual_dist)
    annual_dist['rank'] = annual_dist['Q'].rank(ascending=False)
    annual_dist['b'] = 1 - annual_dist['rank'] / float(len(annual_dist) - 1)
    june_val = annual_dist[annual_dist.index == 6]['b'].values[0]
    if june_val > 0.8:
        return 7
    else:
        return 6

In [ ]:
def train_model(input_data):
    tstart = time.time()
    training_months = input_data['months']
    training_year = input_data['year']
    wsc_station_num = input_data['wsc_stn']
    training_sample_size = input_data['n_sample']
    stn_da = input_data['stn_da']
    closest_radar_stn = input_data['radar_stn']
    
    lag_df = input_data['lag_df']
    num_lags = input_data['num_lags']
    runoff_df = input_data['runoff_df']
    
    dataset_train, dataset_test = split_train_and_test_data(lag_df, training_months, training_year)
    
    training_set_len = len(dataset_train)
    
    if len(dataset_train) < 25:
#         print('exited because dataset_train is too small')
#         print(dataset_train)
        return pd.DataFrame([]), 0

    t0 = time.time()

    scaler = preprocessing.MinMaxScaler()

    X_train = pd.DataFrame(scaler.fit_transform(dataset_train), 
                                  columns=dataset_train.columns, 
                                  index=dataset_train.index)
    # Random shuffle training data
    X_train.sample(frac=1)

    X_test = pd.DataFrame(scaler.transform(dataset_test), 
                                 columns=dataset_test.columns, 
                                 index=dataset_test.index)
    t1 = time.time()
    
   
    X_train_PCA, X_test_PCA, var_expl, n_components = do_PCA(X_train, X_test, num_lags)
    
    data_train = np.array(X_train_PCA.values)
    data_test = np.array(X_test_PCA.values)
    t2 = time.time()
#     print('time to end of PCA = {:.4f}'.format(t2-tstart))

    
    def cov_matrix(data, verbose=False):
        covariance_matrix = np.cov(data, rowvar=False)
        if is_pos_def(covariance_matrix):
            inv_covariance_matrix = np.linalg.inv(covariance_matrix)
            if is_pos_def(inv_covariance_matrix):
                return True, covariance_matrix, inv_covariance_matrix
            else:
                print("Error: Inverse of Covariance Matrix is not positive definite!")
                return False, None, None
        else:
#             print("Error: Covariance Matrix is not positive definite!")
            return False, None, None

               
    cov_test, cov_matrix, inv_cov_matrix = cov_matrix(data_train)
    
    if cov_test == False:
        return pd.DataFrame([]), 0

    mean_distr = data_train.mean(axis=0)

    dist_test = MahalanobisDist(inv_cov_matrix, mean_distr, data_test, verbose=False)
    dist_train = MahalanobisDist(inv_cov_matrix, mean_distr, data_train, verbose=False)
    threshold = MD_threshold(dist_train, extreme = True)
    
    anomaly_train = pd.DataFrame()
    anomaly_train['Mob dist']= dist_train
    anomaly_train['Thresh'] = threshold
    # If Mob dist above threshold: Flag as anomaly
    anomaly_train['Anomaly'] = anomaly_train['Mob dist'] > anomaly_train['Thresh']
    anomaly_train.index = X_train_PCA.index
    anomaly = pd.DataFrame()
    anomaly['Mob dist']= dist_test
    anomaly['Thresh'] = threshold
    anomaly['num_components_kept'] = n_components
    # If Mob dist above threshold: Flag as anomaly
    anomaly['Anomaly'] = anomaly['Mob dist'] > anomaly['Thresh']
    anomaly.index = X_test_PCA.index
    anomaly.head()
    
    anomaly_alldata = pd.concat([anomaly_train, anomaly], sort=True)
    
    event_times = np.where(anomaly_alldata['Anomaly'].values[:-1] != anomaly_alldata['Anomaly'].values[1:])[0]
    events = pd.merge(lag_df, anomaly_alldata.iloc[event_times,:], how='inner', 
                      left_index=True, right_index=True)

    events = events.loc[~events.index.duplicated(keep='first')]
    
    if len(events) < 5:
#         print('exited because len(events) < 5')
        return pd.DataFrame([]), n_components
    elif events.iloc[0]['Anomaly'] == True:
        events = events.iloc[1:]
        
    # create a column of time difference between events in days
    events['dt_days'] = events.index.to_series().diff(1)    

    a = time.time()

    last_event_end = False

    new_events = pd.DataFrame()

    # iterate through the detected event pairs 
    for i in np.arange(0, len(events) - 1, 2):
        
        # parse a single event pair
        this_event = events.iloc[i:i+2]
        
        check_sign_switch = this_event['Anomaly'].values[0] != this_event['Anomaly'].values[1]
        taa = time.time()
        concurrent_wsc = lag_df[(lag_df.index >= this_event.index.values[0]) & (lag_df.index <= this_event.index.values[1])][['Q']]
        peak_in_middle = check_peak_in_middle(this_event, concurrent_wsc)


        if (check_sign_switch) & (peak_in_middle):
            
            
            
            # get the start date
            this_event_start = pd.to_datetime(this_event[this_event['Anomaly'] == False].index.values[0])
            # get the end date
            this_event_end = pd.to_datetime(this_event[this_event['Anomaly'] == True].index.values[0])
            tloops = time.time()
            adjusted_start_date = pd.to_datetime(adjust_edge_date(this_event_start, lag_df[['Q']], 'start', stn_da))
            adjusted_end_date = pd.to_datetime(adjust_edge_date(this_event_end, lag_df[['Q']], 'end', stn_da))
            
            lag_df_start = pd.to_datetime(lag_df.index.values[0])
            
            tin = time.time()
#             print('asd {:.3f}'.format(tin - tloops))
            
            # check if the adjusted start date predates the record
            if lag_df_start > adjusted_start_date:
                adjusted_start_date = lag_df_start
#                 print('this was adjusted')
            
            if last_event_end is not False:
                # find if the start date is on the rising limb - adjust if so

                if adjusted_start_date < last_event_end:
                    adjusted_start_date = last_event_end + pd.DateOffset(1)
                    
            new_event_start = lag_df[lag_df.index == adjusted_start_date][['Q']]
            new_event_end = lag_df[lag_df.index == adjusted_end_date][['Q']] 

            new_event_start['timing'] = 'start'
            new_event_end['timing'] = 'end'
            
            start_month_limit = get_start_from_annual_distribution(runoff_df)            
            
            if stn_da < 100:
                max_days = 4
            elif stn_da < 500:
                max_days = 6
            else:
                max_days = 14
                
            if len(new_event_start) == 0:
                new_event_start = lag_df[lag_df.index == this_event_start][['Q']]
            if len(new_event_end) == 0:
                new_event_end = lag_df[lag_df.index == this_event_end][['Q']]
            
            min_time_check = (new_event_end.index - new_event_start.index).days > 1
            max_time_check = (new_event_end.index - new_event_start.index).days <= max_days
            start_month = new_event_start.index.month
            
            end_month = new_event_end.index.month
            season_check = (start_month > 5) & (start_month < 11) & (end_month <= 11)

            if (min_time_check) & (max_time_check) & (season_check):
                new_events = new_events.append(new_event_start, sort=True)
                new_events = new_events.append(new_event_end, sort=True)

            last_event_end = pd.to_datetime(this_event_end)
            

    new_events.sort_index(inplace=True)    

    new_events['dt_days'] = new_events.index.to_series().diff(1)
    new_events['wsc_station'] = wsc_station_num
    new_events['training_year'] = training_year
    new_events['training_months'] = str(training_months)# for e in new_events]
    new_events['training_set_len'] = training_set_len
    new_events['m_threshold'] = threshold
    new_events['var_explained'] = var_expl
    new_events['n_components'] = n_components
    new_events['num_lags'] = num_lags
    new_events['radar_stn'] = closest_radar_stn
                
    return new_events, n_components

In [ ]:
def adjust_edge_date(initial_date, data, direction, stn_da):
    """
    If the start flow is on a rising limb, adjust the start to the start of the runoff event.
    """
    initial_val = data[data.index == initial_date]['Q']

    
    if direction == 'start':
        search_criteria = (data.index >= initial_date - pd.Timedelta('7 days')) & (data.index <= initial_date)
        search_direction = 1
    elif direction == 'end':
        search_criteria = (data.index <= initial_date + pd.Timedelta('3 days')) & (data.index >= initial_date)
        search_direction = 1

        
    extended_week_vals = data[search_criteria][['Q']]
    extended_week_vals['diff'] = extended_week_vals.diff(periods=search_direction)
    extended_week_vals['pct_change'] = 100 * extended_week_vals['diff'] / extended_week_vals['Q']

    if direction == 'start':
        try:
            change_date = pd.to_datetime(extended_week_vals[['Q']].idxmin().values[0])
            change_point_date = change_date - pd.DateOffset(1)
            adjusted_date = change_point_date
            
        except ValueError as err:
            adjusted_date = initial_date

    elif direction == 'end':
        try:
            adjusted_date = pd.to_datetime(extended_week_vals[['diff']].idxmin().values[0])
        except ValueError as err:
            print('print error in adjusting event end date', err)
            adjusted_date = initial_date
            
    return adjusted_date


def check_peak_in_middle(event, data):
    """
    Ensure there is a peak between the start and end points
    so we aren't targeting a non-runoff event.
    """
    start_time = event.index.values[0] 
    end_time = event.index.values[-1]
    max_time = data[data['Q'] == data['Q'].max()].index.values[0]
    if (max_time == start_time) | (max_time == end_time):
        return False
    else:
        return True


def get_all_combinations(months, years):
    month_combos = [list(itertools.combinations(months, n)) for n in list(range(1, 13))]
    flat_combos =  [item for sublist in month_combos for item in sublist]
    return np.asarray(list(itertools.product(flat_combos, years)))

def calc_softmax(X):
    return np.exp(X) / np.sum(np.exp(X))

def filter_input_data(data):
    filtered = []
    for d in data:
        months = list(d[0])
        common_months = [m for m in months if m in [7, 8, 9, 10]]
        if (len(months) == 1) & (months[0] not in [12, 1, 2, 3]):
            filtered.append(list(d))
        elif (len(months) > 1) & (len(common_months) > 0):
            filtered.append(list(d))
    return filtered

def run_AD_training(wsc_station_num, training_sample_size=5):
    
    radar_stn = stn_df[stn_df['Station Number'] == wsc_station_num]['closest_radar_station'].values[0]
    stn_da = stn_df[stn_df['Station Number'] == wsc_station_num]['DA'].values[0]
    
    lag_df, closest_radar_stn, runoff_df, num_lags = initialize_input_data(wsc_station_num)
    
#     runoff_df = initialize_runoff_dataframe(wsc_station_num)  
    
    training_months = list(set(runoff_df.index.month))
    training_years = list(set(runoff_df.index.year))

    all_combinations = get_all_combinations(training_months, training_years)
    
    filtered_combinations = np.array(filter_input_data(all_combinations))
        
    weights = calc_softmax([(13.0 - len(c[0]))*3.5 for c in filtered_combinations])
    
    # a complete search is intractable, so sample n permutations without replacement
    rand_ints = np.random.choice(range(len(filtered_combinations)), training_sample_size, 
                                 replace=False, p=weights)
    
    initial_pop = [filtered_combinations[i] for i in rand_ints]
    

    input_array = []
    for combo in initial_pop:
        input_data = {'year': combo[1],
                     'months': combo[0],
                     'n_sample': training_sample_size,
                     'wsc_stn': wsc_station_num,
                     'stn_da': stn_da,
                     'radar_stn': radar_stn,
                      'lag_df': lag_df,
                      'num_lags': num_lags,
                      'runoff_df': runoff_df
                     }
        input_array.append(input_data)
        
    results = []
    for input_dat in input_array:
        result, n_components = train_model(input_dat)
        results.append((input_dat, n_components, result))
    
    return results


In [ ]:
def initialize_input_data(wsc_stn_num):
        
    t0 = time.time()
    stn_df = initialize_wsc_station_info_dataframe()

    test_stn_info = stn_df[stn_df['Station Number'] == wsc_stn_num]
    stn_da = test_stn_info['DA'].values[0]
    wsc_stn_name = test_stn_info['Station Name'].values[0]
    closest_radar_stn = test_stn_info['closest_radar_station'].values[0]
#     print('{} ({}) has a DA of {} km^2'.format(wsc_stn_name, wsc_stn_num, stn_da))
    
    runoff_df = initialize_runoff_dataframe(wsc_stn_num)    
    lag_df, num_lags = create_lag_df(runoff_df, stn_da) 
    
    
    candidate_stations = stn_df['Station Number'].values
    
    return lag_df, closest_radar_stn, runoff_df, num_lags

## Code to test a single station and view AD results

In [ ]:
def run_AD_from_results(test_stn):

    test_flow_df = get_daily_runoff(test_stn)
    test_flow_df = test_flow_df[test_flow_df.index.year >= 2007]
    test_flow_df.rename(columns={'DAILY_FLOW': 'Q'}, inplace=True)
    test_flow_df = test_flow_df[['Q']]
    
    radar_stn = stn_df[stn_df['Station Number'] == test_stn]['closest_radar_station'].values[0]
    stn_da = stn_df[stn_df['Station Number'] == test_stn]['DA'].values[0]
    
    lag_df, closest_radar_stn, runoff_df, num_lags = initialize_input_data(test_stn)
    
    best_training_params = get_best_result(test_stn)

    months = [m for m in best_training_params[0][1:-1].split(',') if len(m) > 0]
    train_months = [int(m) for m in months]
       
    input_data = {'year': best_training_params[1],
             'months': train_months,
             'n_sample': best_training_params[4],
             'wsc_stn': test_stn,
             'stn_da': stn_da,
             'radar_stn': radar_stn,
              'lag_df': lag_df,
              'num_lags': num_lags,
              'runoff_df': runoff_df
            }
        
    best_events, n_components = train_model(input_data)

    return best_events, test_flow_df, closest_radar_stn

test_stn = '05CA012'
# print(best_training_params)
best_events, test_flow_df, closest_radar_stn = run_AD_from_results(test_stn)
print(best_events)

In [ ]:
p = figure(plot_width=800, plot_height=400, x_axis_type='datetime')

e1 = best_events[best_events['timing'] == 'start']
e2 = best_events[best_events['timing'] == 'end']

p.circle(e1.index, e1['Q'], color='red', alpha=0.5, size=10, legend_label='start')
p.circle(e2.index, e2['Q'], color='blue', alpha=0.5, size=10, legend_label='end')

# p.line(input_sig.index, input_sig['f_sig'], color='blue')
p.line(test_flow_df.index, test_flow_df['Q'], color='blue')
# p.line()
# show the results
show(p)

In [ ]:
# create grid plot of individual events

plots = []

for i in np.arange(0, len(best_events) - 1, 2):
    
    # parse a single event pair
    this_event = best_events.iloc[i:i+2]
    
    s1 = figure(background_fill_color="#fafafa", x_axis_type='datetime')
    
    s1.circle(this_event.index, this_event['Q'], 
              size=12, alpha=0.8, color="red")#, legend_label='{estimated endpoints}')
    s1.xaxis.major_label_orientation = math.pi/2
    this_start = pd.to_datetime(this_event.index.values[0])
    this_end = pd.to_datetime(this_event.index.values[1])
    this_dat = test_flow_df[(test_flow_df.index >= this_start) & (test_flow_df.index <= this_end)][['Q']]
    
    if (this_end.month < 12) & (this_start.month > 5):
        year = this_event.index.year.values[0]
        month = this_event.index.month.values[0]
        day = this_event.index.day.values[0]
        date = '{}-{}-{}'.format(year, month, day)
        s1.line(this_dat.index, this_dat['Q'], color='blue')
        plots.append(s1)

print('there are {} plots'.format(len(plots)))

In [ ]:
# if len(plots) < 6:
#     grid = gridplot(plots, plot_width=150, plot_height=150)
# else:
n_cols = 8
n_rows = int(np.ceil(len(plots) / n_cols))


g = []
for i in range(0, len(plots), n_cols):
    g += [plots[i:i+n_cols]]
grid = gridplot(g, plot_width=150, plot_height=150)

In [ ]:
show(grid)

## Code to batch process all station events and aquire corresponding radar images

In [ ]:
def get_all_event_pairs(best_events):
    event_pairs = []
    for i in np.arange(0, len(best_events) - 1, 2):
        # parse a single event pair
        this_event = best_events.iloc[i:i+2]
        date_pair = [e.astype(str).replace('T', ' ').split('.')[0].split(' ')[0] for e in this_event.index.values]
        this_start = pd.to_datetime(this_event.index.values[0])
        this_month = this_start.month
        if (this_month > 5) & (this_month <= 11) & (this_start > pd.to_datetime('2007-01-01')):
            event_pairs.append(date_pair)
    return event_pairs

In [ ]:
i = 0
for stn in all_sites: 
    i += 1
    print(stn)
    best_events, test_flow_df, closest_radar_stn = run_AD_from_results(stn)
    event_pairs = get_all_event_pairs(best_events)
    all_img_urls = get_radar_img_urls(event_pairs, closest_radar_stn)
    n_images = len(all_img_urls)
    if n_images > 0:
        print('{} of {}: {} image files to query for {} with {} events identified.'.format(i, len(all_sites), 
                                                                                           len(all_img_urls), stn, 
                                                                                           len(event_pairs)))
    time0 = time.time()    
    request_img_files(all_img_urls, stn, closest_radar_stn)
    time1 = time.time()
    print('    {:.0f}s to query all images'.format(time1 - time0))

### Load Radar, Mask with Catchment, and Clip all Retrieved Images

First, make sure basin geometry is available before requesting all images.

# TODO:



## Further Work

Find all the stations that have radar overlap.  

Find a bunch of concurrent runoff events, and estimate the error based on the overlapping radar info.

In [ ]:
def get_ubc_polygon():
    points = [-123.1876189229259,49.27349631820059, -123.2168376964195,49.27886813665509,
              -123.250951627253,49.28123248845857, -123.2686880871375,49.26628206996413, 
              -123.2577985144227,49.25107796072373, -123.2296736741512,49.23345938143058, 
              -123.2086850019333,49.22474925352834, -123.1911799345363,49.22020024319045, 
              -123.1852881427934,49.2196036807795, -123.1876189229259,49.27349631820059, ]
    lat_list = []
    lon_list = []
    for i in range(0, len(points), 2):
        lat_list.append(points[i])
        lon_list.append(points[i + 1])
#     gdb_path = os.path.join(PROJECT_DIR, path)
#     data = gpd.read_file(gdb_path, driver='kmz')
#     return data.geometry
    minx = min(lat_list)
    maxx = max(lat_list)
    miny = min(lon_list)
    maxy = max(lon_list)
    bbox = pd.DataFrame({'minx': [minx], 'miny': [miny],
                        'maxx': [maxx], 'maxy': [maxy]})
    return Polygon(zip(lat_list, lon_list)), bbox

In [ ]:
import shapely.speedups
shapely.speedups.enable()

In [ ]:
def get_pixel_coordinates(closest_stn):
    # note that the pixel coordinates are projected to NAD83
    # coordinate system by assuming each pixel is 1 / 111.32 
    # degree seconds, so the error propagates from the centre
    # to 0.01 * 240 = 2.4 metres at the edges of the image
    img_coord_path = 'data/radar_img_pixel_coords'
    img_coord_file = [f for f in os.listdir(img_coord_path) if closest_stn in f][0]
    geo_df = gpd.read_file(os.path.join(img_coord_path, img_coord_file))
    geo_df = geo_df.to_crs('EPSG:4269')
#     return pd.read_json(os.path.join(img_coord_path, img_coord_file))
    return geo_df


def get_img_mask(closest_stn, basin_geom, wsc_stn):
    # take in the basin geometry and its corresponding radar station
    # return a 480x480 boolean matrix where True represents
    # pixels that fall within the basin polygon
    # note that the points are stored in (lat, lon) tuples
    # which corresponds to y, x
    basin_geom = basin_geom.to_crs('EPSG:4326')
    print('basin geom:', basin_geom.crs)
#     basin_geom_data = basin_data.geometry
    print(basin_geom.bounds)
#     print(basin_geom_data)
    
    mask_folder = 'data/wsc_stn_basin_masks'
    mask_path = os.path.join(PROJECT_DIR, mask_folder)
    mask_filename = wsc_stn + '.json'
    existing_masks = os.listdir(mask_path)

    if mask_filename in existing_masks:
        return pd.read_json(os.path.join(mask_path, mask_filename)).to_numpy()
    else:
        print('No image mask exists.  Try creating one...')
        geo_df = get_pixel_coordinates(closest_stn)#.to_numpy().flatten()

    # match the crs of the basin polygon
    geo_df = geo_df.to_crs('EPSG:4326')
    print(basin_geom.geometry)
#     geo_df = geo_df.to_crs('EPSG:4269')
    
#     print('geo df bounds:')
#     print(geo_df.total_bounds)
#     print(geo_df.crs)
#     print('catchment geom bounds:')
#     print(basin_geom.bounds)
#     print('')

    pip_mask = geo_df.within(basin_geom.geometry)
    
    reshaped_mask = np.array(pip_mask).reshape(480, 480)

    img_mask_df = pd.DataFrame(reshaped_mask)

    img_mask_df.to_json(os.path.join(mask_path, mask_filename), orient='columns')
    
    print('saved json')


def bbox2(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    return img[ymin:ymax+1, xmin:xmax+1]


@jit
def mask_image(img_array, mask):
    mask_idx = np.where(mask==0)
    rows = mask_idx[0]
    cols = mask_idx[1]
    filtered_img = np.zeros((480, 480, 3))
    for n in range(480):
        for m in range(480):
            if mask[n, m] == 0:
                filtered_img[n, m] = 0
            else:
                filtered_img[n, m] = img_array[n, m]
    return filtered_img

In [ ]:
def mask_images_and_save(gif_images, mask, test_stn, best_radar_stn):
#     print(best_radar_stn)
    origin_path = os.path.join(RADAR_IMG_DIR, best_radar_stn)
    save_path = os.path.join(PROJECT_DIR, 'data/masked_img/{}'.format(test_stn))
    for t_img in gif_images:
        new_filename = t_img.split('.')[0] + '_crp.gif'
        time0 = time.time()      
        origin_full_path = os.path.join(origin_path, t_img)
        new_file_path = os.path.join(save_path, new_filename)
        # there are a large number of empty images that get returned
        # from image retrievel.  DON'T DELETE THEM or you will 
        # request them many times over in the AD function.
        # Instead, filter them out here.
        if os.path.getsize(origin_full_path) > 100:
            gif_img = Image.open(origin_full_path).convert('RGB')
            img_array = np.asarray(gif_img)[:,:480].astype(np.uint8)        
            filtered_img = mask_image(img_array, mask)
            time1 = time.time()
            try:
                cropped_array = bbox2(filtered_img)
                cropped_img = Image.fromarray(cropped_array.astype(np.uint8))
                cropped_img.save(os.path.join(save_path, new_filename))
            except Exception as e:
                print('Something broke in the image mask process for {}'.format(test_stn))
                print(e)
                break


def get_event_times(df):
    event_pairs = []
    for i in np.arange(0, len(df) - 1, 2):    
        # parse a single event pair
        this_event = df.iloc[i:i+2]
        date_pair = [e.astype(str).replace('T', ' ').split('.')[0].split(' ')[0] for e in this_event.index.values]
        this_start = pd.to_datetime(this_event.index.values[0])
        this_month = this_start.month
        if (this_month > 5) & (this_month <= 11) & (this_start > pd.to_datetime('2007-01-01')):
            event_pairs.append(date_pair)
    return event_pairs
    
    
def get_possible_filenames_from_event_time(event_pair):
    filenames = []
    for p in event_pair:
        start, end = p[0], p[1]
        datetime_range = [str(pd.to_datetime(e).strftime('%Y%m%d%H%M')) + '.gif' for e in pd.date_range(pd.to_datetime(start), 
                                       pd.to_datetime(end),
                                      freq='1H').values]
        filenames += datetime_range
    return set(filenames)


def get_radar_images_for_events(wsc_stn, radar_stn, best_events):
    event_df = best_events
    event_pairs = get_event_times(event_df)
    
    files_to_search = get_possible_filenames_from_event_time(event_pairs)
    
    existing_files = os.listdir(os.path.join(RADAR_IMG_DIR, radar_stn))
    
#     new_files = np.setdiff1d(files_to_search, existing_files)
    new_files = files_to_search.intersection(existing_files)
    return new_files


def get_gif_images(test_stn, closest_radar_stn, best_events):
    gif_path = 'data/sorted_radar_images/{}'.format(closest_radar_stn)
    save_path = 'data/masked_img/{}'.format(test_stn)
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    return list(get_radar_images_for_events(test_stn, closest_radar_stn, best_events))

In [ ]:
def get_basin_geometry(test_stn):
    # original WSC basin polygon is EPSG: 4269 (NAD83)
    # WGS 84 is EPSG: 4326
    basin_data = basin_df[basin_df['Station'] == test_stn]
    return basin_data

def calculate_distance(row):
    wsc_stn_coords = [row['Latitude'], row['Longitude']]
    radar = row['Closest_radar']
    
    radar_coords = radar_stations[radar]['lat_lon']
    xs = [wsc_stn_coords[1], radar_coords[1]]
    ys = [wsc_stn_coords[0], radar_coords[0]]

    geom = [Point(xy) for xy in zip(xs, ys)]
    gdf = gpd.GeoDataFrame(geometry=geom, crs='epsg:4269')
    gdf.to_crs(epsg=3310, inplace=True)
    # calculate distance in kilometers
    l = gdf.distance(gdf.shift()).values[-1] / 1000
    return l



In [ ]:
def create_mask(wsc_stn):
    
    best_events, test_flow_df, closest_radar_stn = run_AD_from_results(wsc_stn)
    event_pairs = get_all_event_pairs(best_events)
    
#     print('closest radar = ', closest_radar_stn)
    
    stn_da = stn_df[stn_df['Station Number'] == wsc_stn]['DA'].values[0]
    
    try:
        basin_geom = get_basin_geometry(wsc_stn)
    except ValueError as err:
        print('No basin geometry available for {}.'.format(wsc_stn))
        return None
#     closest_radar_stn = stn_df[stn_df['Station Number'] == wsc_stn]['Closest_radar'].values[0]
    if len(basin_geom) > 0:
        t0 = time.time()
        get_img_mask(closest_radar_stn, basin_geom, wsc_stn)
        t1 = time.time()
        print('Image mask retrieved in {:.1f} s.'.format(t1 - t0))
    else:
        return None


## Create image masks for all WSC stations

It takes between 1-2 minutes to create the station mask, so loading them once 
and pre-saving saves considerable time.

In [ ]:
# this masking process takes a really long time because 
# producing the masked array is really slow.
# should try finding a better algorithm but 'union' doesn't work.
# Maybe there's a workaround?
# i = 0
# for stn in all_sites[138:]:
#     i += 1
#     stn_name = stn_df[stn_df['Station Number'] == stn]['Station Name'].values[0]
#     print('{} of {}: Starting image masking function on station {}: {}'.format(i, len(all_sites),
#                                                                                stn, stn_name))

    
#     create_mask(stn)

    

In [ ]:
def run_masking_function(wsc_stn):    
    
    best_events, test_flow_df, closest_radar_stn = run_AD_from_results(wsc_stn)
    if len(best_events) > 0:

        event_pairs = get_all_event_pairs(best_events)


    #     best_result = get_best_result(wsc_stn)
    #     runoff_df = initialize_runoff_dataframe(stn)
        stn_da = stn_df[stn_df['Station Number'] == wsc_stn]['DA'].values[0]

        gif_images = get_gif_images(wsc_stn, closest_radar_stn, best_events)
        time0 = time.time()

        mask_folder = 'data/wsc_stn_basin_masks'
        mask_path = os.path.join(PROJECT_DIR, mask_folder)
        mask_filename = wsc_stn + '.json'
        existing_masks = os.listdir(mask_path)
        if mask_filename in existing_masks:
            try:
                mask = pd.read_json(os.path.join(mask_path, mask_filename)).to_numpy()
            except ValueError as ve:
                print(wsc_stn)
                print(ve)

            mask_images_and_save(gif_images, mask, wsc_stn, closest_radar_stn)
            time1 = time.time()
            print('    Time to process {} images for {}: {:.1f}s, ({:.2f}s/image)'.format(len(gif_images), 
                                                                                      wsc_stn,
                                                                                       time1-time0, 
                                                                                       (time1-time0)/len(gif_images)))
        else:
            print('    No mask available for {}.'.format(wsc_stn))
    else:
        print('    No results returned for {}.'.format(wsc_stn))


In [ ]:
# print(stn_df[stn_df['Station Number'] == '08NE039'])

In [ ]:
i = 1
for stn in all_sites[1:]:
    print('starting {} of {}: {} ...'.format(i, len(all_sites), stn))
    run_masking_function(stn)
    i += 1

In [ ]:
# ubc_polygon = get_ubc_polygon()

In [ ]:
# mask = get_img_mask(best_radar_stn, basin_geom)
# mask = get_img_mask('CASSS', )
# mask = get_img_mask('CASAG', ubc_polygon)

In [ ]:
gif_images = get_gif_images(test_stn, best_radar_stn)
# gif_images = os.listdir(os.path.join(RADAR_IMG_DIR, 'CASAG'))

In [ ]:
# open the image file

# gif_img = Image.open('data/radar_img/08HF006/200708261000.gif') # 640x480x3 array
# label = gif_img.convert('RGB')
# img_array = np.asarray(label)
# radar_img_array = np.asarray(img_array)[:,:480]
# color_bar_array = img_array[144:340, 515:535]
# color_bar_img = Image.fromarray(color_bar_array, mode='RGB')
# colors = color_bar_img.getcolors()

| Item | Default Unit | Other Possible Conversions |
|---|---|---|
| Station Loc. | Lat/Lon | UTM (with zone) | None |
| Catchment Boundary | ESRI:102001 | Anything Geopandas can do |
| Satellite Radar Img. | None | Radar centre can have UTM, pixels are 1kmx1km | 

In [ ]:
# format WSC station data points into a geodataframe
all_stations = stn_df['Station Number'].values

geo_df = stn_df[['Station Number', 'Latitude', 'Longitude']]
# geo_df.crs = geo_df.'EPSG:4326'
wsc_sites_geo = gpd.GeoDataFrame(geo_df, geometry=gpd.points_from_xy(geo_df['Longitude'], geo_df['Latitude']),
                                crs='EPSG:4326')
# Convert to Mercator for Plotting
wsc_sites_geo = wsc_sites_geo.to_crs("EPSG:3857")

# Get x and y coordinates
wsc_sites_geo['x'] = [geometry.x for geometry in wsc_sites_geo['geometry']]
wsc_sites_geo['y'] = [geometry.y for geometry in wsc_sites_geo['geometry']]
wsc_df = wsc_sites_geo.drop('geometry', axis = 1).copy()
sitesource = ColumnDataSource(wsc_df)

In [ ]:
# format radar station data points into a geodataframe

r_df = pd.DataFrame(radar_stations).T
r_df['Latitude'] = [e[0] for e in r_df['lat_lon']]
r_df['Longitude'] = [e[1] for e in r_df['lat_lon']]
r_df = r_df[['alt_name', 'Latitude', 'Longitude']]
r_df_geo = gpd.GeoDataFrame(r_df, geometry=gpd.points_from_xy(r_df['Longitude'], r_df['Latitude']),
                                crs='EPSG:4326')
# convert to Mercator for plotting
# r_sites_geo = r_df_geo.to_crs("EPSG:3395")
r_sites_geo = r_df_geo.to_crs("EPSG:3857")

# Get x and y coordinates
r_sites_geo['x'] = [geometry.x for geometry in r_sites_geo['geometry']]
r_sites_geo['y'] = [geometry.y for geometry in r_sites_geo['geometry']]

rad_df = r_sites_geo.drop('geometry', axis = 1).copy()
radar_sitesource = ColumnDataSource(rad_df)

In [ ]:
# find bounding box for BC and alberta stations
# note Mercator projection is EPSG:3857

l_box, r_box = wsc_df['x'].min(), wsc_df['x'].max()
t_box, b_box = wsc_df['y'].max(), wsc_df['y'].min()

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.tile_providers import Vendors, get_provider
import utm

# tile_provider = get_provider(Vendors.CARTODBPOSITRON)
tile_provider = get_provider(Vendors.STAMEN_TERRAIN_RETINA)

# print(bbox)
# range bounds supplied in web mercator coordinates
p = figure(x_range=(l_box*1.0001, 1.0001*r_box), y_range=(0.98*b_box, 1.01*t_box),
           x_axis_type="mercator", y_axis_type="mercator",
          width=800, height=400)

p.add_tile(tile_provider)

p.diamond('x', 'y', source=radar_sitesource, color='black', 
                 size=8, alpha=0.9, legend_label='Radar Stations')
p.circle('x', 'y', source=radar_sitesource, color='blue', 
                 size=5, alpha=0.2, legend_label='Radar Range',
                radius=400000, radius_dimension='max')

p.circle('x', 'y', source=sitesource, color='red', 
                 size=5, alpha=0.3, legend_label='WSC Stations')

show(p)


In [ ]:
basin_geodf = gpd.GeoDataFrame(basin_geom,
                                crs='EPSG:4326')
# convert to Mercator for plotting
basin_geo = basin_geodf.to_crs("EPSG:3857")

# Get x and y coordinates
basin_polygon = list(basin_geo.iloc[0].geometry[0].exterior.coords)

basin_df = pd.DataFrame()
basin_df['x'] = [e[0] for e in basin_polygon]
basin_df['y'] = [e[1] for e in basin_polygon]

basin_sitesource = ColumnDataSource(basin_df)

l_box, r_box = basin_df['x'].min(), basin_df['x'].max()
b_box, t_box = basin_df['y'].min(), basin_df['y'].max()




In [ ]:
print(test_stn)

In [ ]:
p = figure(x_range=(l_box*1.0001, 1.0001*r_box), y_range=(0.98*b_box, 1.01*t_box),
           x_axis_type="mercator", y_axis_type="mercator",
          width=800, height=400)

p.add_tile(tile_provider)

p.line('x', 'y', source=basin_sitesource, color='blue', 
     legend_label='WSC 08HB048 Catchment',
      line_dash='dashed')

p.circle('x', 'y', source=sitesource, color='red', 
                 size=5, alpha=0.3, legend_label='WSC Stations')

p.match_aspect = True
# gdf.plot('geometry', ax=ax, color='lightgray', alpha=0.1)
# basin_geom.plot(ax=ax, alpha=0.35, edgecolor='k', linewidth=2)
# print(stn_df[stn_df['Station Number'] == test_stn])
show(p)

